In [1]:
# necessary imports
import os, json
import pandas as pd
import numpy as np
import glob
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
import sys
sys.path.append('../scripts/')
from twitter import get_hashtags
import ast

In [2]:
# read lda dates for hashtags
dates_df = pd.read_csv('../../data/BTW17_Twitter/peaks/hashtags_dates.csv')
dates_df = dates_df[~(dates_df['lda_dates']=='[]')].reset_index()
dates_df.drop(columns=['Unnamed: 0', 'index'], axis=1, inplace=True)
dates_df.head()

,hashtag,lda_dates
0,afd,"['2017-09-16', '2017-09-17', '2017-09-18', '20..."
1,afghanistan,"['2017-08-22', '2017-08-23', '2017-08-24', '20..."
2,armut,"['2017-08-28', '2017-08-29', '2017-08-30', '20..."
3,bayern,"['2017-09-11', '2017-09-12', '2017-09-13', '20..."
4,berlin,"['2017-09-02', '2017-09-03', '2017-09-04', '20..."


In [3]:
# set to *.json to load all
path_to_json = '../../data/BTW17_Twitter/tweets/*.json'
file_list = glob.glob(path_to_json)
hashtag_list = dates_df['hashtag'].tolist()
lda_dates_list = dates_df['lda_dates'].tolist()


output_df = pd.DataFrame()

# iterate through files create output csv
for index in tqdm(range(len(file_list))):
    file = file_list[index]
    with open(file, 'r') as f:
        
        # read df
        df = pd.read_json(f)
        # apply function get hashtags
        df['tags'] = df['entities'].apply(get_hashtags)
        
        # explode tags to rows and drop na values
        df = df.explode('tags')
        df = df[df['tags'].notna()]
        
        # change datetime to date and lower all hashtags
        df['created_at'] = pd.to_datetime(df['created_at']).dt.date
        df['tags'] = df['tags'].apply(lambda x: str(x).lower())
                
        for i in range(len(dates_df)):
            #hashtag = 'ehrenwertegesellschaft'
            #lda_dates = ['2017-07-25', '2021-01-01', '2022-03-01']
            hashtag = hashtag_list[i]
            lda_dates = pd.to_datetime(ast.literal_eval(lda_dates_list[i]))
            temp_df = df[(df['tags']==hashtag)&(df['created_at'].isin(lda_dates))]
            output_df = output_df.append(temp_df)

In [4]:
output_df.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,entities,extended_entities,favorited,retweeted,possibly_sensitive,filter_level,lang,timestamp_ms,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,extended_tweet,tags,withheld_in_countries,quote_count,reply_count,delete,status_withheld
18,2017-06-28,8.798635e+17,8.798635e+17,RT @Junge_Freiheit: AfD-Vize Beatrix von Storc...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,NaN,None,"{'id': 728326774027919360, 'id_str': '72832677...",NaN,NaN,None,NaN,0.0,0.0,0.0,"{'hashtags': [{'text': 'Ehefueralle', 'indices...",NaN,0.0,0.0,NaN,low,de,2017-06-28 00:46:10.413,{'created_at': 'Tue Jun 27 16:01:11 +0000 2017...,NaN,NaN,NaN,NaN,ehefueralle,NaN,NaN,NaN,NaN,NaN
67,2017-06-28,8.798879e+17,8.798879e+17,RT @AfD_Bund: Dr. @Alice_Weidel:\n»#Ehefuerall...,NaN,"<a href=""http://twitter.com/download/android"" ...",0.0,NaN,NaN,NaN,NaN,None,"{'id': 1222045254, 'id_str': '1222045254', 'na...",NaN,NaN,None,NaN,0.0,0.0,0.0,"{'hashtags': [{'text': 'Ehefueralle', 'indices...",NaN,0.0,0.0,NaN,low,de,2017-06-28 02:23:26.159,{'created_at': 'Tue Jun 27 17:40:08 +0000 2017...,NaN,NaN,NaN,NaN,ehefueralle,NaN,NaN,NaN,NaN,NaN
70,2017-06-28,8.798901e+17,8.798901e+17,"RT @hessenSPD: Abstimmen, abstimmen, abstimmen...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,NaN,None,"{'id': 2443895924, 'id_str': '2443895924', 'na...",NaN,NaN,None,NaN,0.0,0.0,0.0,"{'hashtags': [{'text': 'ehefueralle', 'indices...",NaN,0.0,0.0,NaN,low,de,2017-06-28 02:32:06.046,{'created_at': 'Tue Jun 27 09:19:12 +0000 2017...,NaN,NaN,NaN,NaN,ehefueralle,NaN,NaN,NaN,NaN,NaN
71,2017-06-28,8.798903e+17,8.798903e+17,RT @Die_Gruenen: Hallo @spdbt! Jetzt hindert E...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,NaN,None,"{'id': 12302322, 'id_str': '12302322', 'name':...",NaN,NaN,None,NaN,0.0,0.0,0.0,"{'hashtags': [{'text': 'Ehefueralle', 'indices...",NaN,0.0,0.0,NaN,low,de,2017-06-28 02:32:59.178,{'created_at': 'Tue Jun 27 07:59:48 +0000 2017...,NaN,NaN,NaN,NaN,ehefueralle,NaN,NaN,NaN,NaN,NaN
72,2017-06-28,8.798903e+17,8.798903e+17,"RT @Die_Gruenen: ""Wer die Debatte auch nur hal...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,NaN,None,"{'id': 391983270, 'id_str': '391983270', 'name...",NaN,NaN,None,NaN,0.0,0.0,0.0,"{'hashtags': [{'text': 'Ehefueralle', 'indices...",NaN,0.0,0.0,NaN,low,de,2017-06-28 02:33:00.094,{'created_at': 'Tue Jun 27 20:27:15 +0000 2017...,NaN,NaN,NaN,NaN,ehefueralle,NaN,NaN,NaN,NaN,NaN


In [8]:
# drop unnecessary columns
output_df.drop(columns=['id_str', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id',
                        'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str',
                        'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors',
                        'is_quote_status', 'retweet_count', 'favorite_count', 'extended_entities',
                        'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang',
                        'timestamp_ms', 'retweeted_status', 'quoted_status_id', 'quoted_status_id_str',
                        'quoted_status', 'extended_tweet', 'withheld_in_countries', 'quote_count',
                        'reply_count', 'delete', 'status_withheld'], axis=1, inplace=True)

# save to csv
path_file = '../../data/BTW17_Twitter/lda/lda_tweets.csv'
output_df.to_csv(path_file)